In [1]:
# ALEX: remove unused code
# import time
# start_time = time.time()
import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
exec(os.environ['IREWR_IMPORTS'])
# ALEX: remove plotting
# import matplotlib.pyplot as plt
# import seaborn as sns
from datetime import datetime
# from pandas import Series
# ALEX: remove ML code
# from sklearn.ensemble import RandomForestRegressor

# from subprocess import check_output

In [2]:
train = pd.read_csv("./input/train.scaled.csv")
test = pd.read_csv("./input/test.scaled.csv")


In [3]:
train.isnull().sum().sum()

0

In [4]:
train.columns

Index(['id', 'vendor_id', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag',
       'trip_duration'],
      dtype='object')

In [5]:
test.columns

Index(['id', 'vendor_id', 'pickup_datetime', 'passenger_count',
       'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'store_and_fwd_flag'],
      dtype='object')

In [6]:
month = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).month
day_of_week = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).weekday()
day = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).day
hour = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).hour
minute = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).minute

seasons = [0,0,1,1,1,2]
season = lambda x: seasons[(datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).month-1)]

In [7]:
train['month'] = train['pickup_datetime'].map(month)
train['day'] = train['pickup_datetime'].map(day)
train['day_of_week'] = train['pickup_datetime'].map(day_of_week)
train['hour'] = train['pickup_datetime'].map(hour)
train['minute'] = train['pickup_datetime'].map(minute)
train['season'] = train['pickup_datetime'].map(season)

In [8]:
train['store_and_fwd_flag']=train['store_and_fwd_flag'].map( {'N': 0, 'Y': 1} ).astype(int)

In [9]:
test['month'] = test['pickup_datetime'].map(month)
test['day_of_week'] = test['pickup_datetime'].map(day_of_week)
test['day'] = test['pickup_datetime'].map(day)
test['hour'] = test['pickup_datetime'].map(hour)
test['minute'] = test['pickup_datetime'].map(minute)
test['season'] = test['pickup_datetime'].map(season)


In [10]:
test['store_and_fwd_flag']=test['store_and_fwd_flag'].map( {'N': 0, 'Y': 1} ).astype(int)


In [11]:
# ALEX: remove plotting
# plt.figure(figsize=(12,5))
# plt.subplot(1, 2, 1)
# Series(train['month']).value_counts().sort_index().plot(kind = 'bar')
# plt.xlabel("Month")
# plt.ylabel("Count")
# plt.title('Which month has the most rides? (train data)')
# plt.subplot(1, 2, 2)
# Series(test['month']).value_counts().sort_index().plot(kind = 'bar')
# plt.xlabel("Month")
# plt.ylabel("Count")
# plt.title('Which month has the most rides? (test data)')
pd.Series(train['month']).value_counts().sort_index()
pd.Series(test['month']).value_counts().sort_index()

1     97676
2    102314
3    109697
4    107432
5    107570
6    100445
Name: month, dtype: int64

In [12]:
# ALEX: remove plotting
# plt.figure(figsize=(12,5))
# plt.subplot(1, 2, 1)
# Series(train['day']).value_counts().sort_index().plot(kind = 'bar')
# plt.xlabel("Day")
# plt.ylabel("Count")
# plt.title('Which day has the most rides?(train data)')
# plt.subplot(1, 2, 2)
# Series(test['day']).value_counts().sort_index().plot(kind = 'bar')
# plt.xlabel("Day")
# plt.ylabel("Count")
# plt.title('Which day has the most rides?(test data)')
pd.Series(train['day']).value_counts().sort_index()
pd.Series(test['day']).value_counts().sort_index()

1     20203
2     20503
3     20828
4     21002
5     21540
6     21392
7     20881
8     20862
9     21308
10    20698
11    21011
12    21200
13    21097
14    21672
15    21202
16    21686
17    21090
18    20584
19    21106
20    21070
21    20790
22    20626
23    17786
24    18974
25    19861
26    20097
27    20037
28    19687
29    19834
30    16594
31     9913
Name: day, dtype: int64

In [13]:
# ALEX: remove plotting
# plt.figure(figsize=(12,5))
# plt.subplot(1, 2, 1)
# Series(train['day_of_week']).value_counts().sort_index().plot(kind = 'bar')
# plt.xlabel("Day of week")
# plt.ylabel("Count")
# plt.title('Which day of week has the most rides? \n (0 = Monday)-(train data)')
# plt.subplot(1, 2, 2)
# Series(test['day_of_week']).value_counts().sort_index().plot(kind = 'bar')
# plt.xlabel("Day of week")
# plt.ylabel("Count")
# plt.title('Which day of week has the most rides? \n (0 = Monday)-(test data)')
pd.Series(train['day_of_week']).value_counts().sort_index()
pd.Series(test['day_of_week']).value_counts().sort_index()

0    80388
1    87318
2    89888
3    93460
4    96260
5    94232
6    83588
Name: day_of_week, dtype: int64

In [14]:
# ALEX: remove extra dependencies
# from geopy.distance import great_circle
def distance(p_lat,p_long,d_lat,d_long):
    pickup = (p_lat, p_long)
    dropoff = (d_lat, d_long)
# ALEX: remove extra dependencies
#     distance_all=great_circle(pickup, dropoff).km
#     return distance_all
    return 50

In [15]:
#for train data
d_train = [distance(train['pickup_latitude'].values[i], train['pickup_longitude'].values[i],
              train['dropoff_latitude'].values[i],train['dropoff_longitude'].values[i]) for i in range(len(train['pickup_latitude']))]
train['distance_est']=d_train

## STEF-UPDATED

In [16]:
train["d1"]=train['distance_est']<=5
# train["d2"]=train['distance_est'].between(5, 10, inclusive=False)
# train["d3"]=train['distance_est'].between(10, 20, inclusive=False)
# train["d4"]=train['distance_est'].between(20, 30, inclusive=False)
# train["d5"]=train['distance_est'].between(30, 100, inclusive=False)
train["d2"]=train['distance_est'].between(5, 10, inclusive="neither")
train["d3"]=train['distance_est'].between(10, 20, inclusive="neither")
train["d4"]=train['distance_est'].between(20, 30, inclusive="neither")
train["d5"]=train['distance_est'].between(30, 100, inclusive="neither")
train["d6"]=train['distance_est']>100

/tmp/ipykernel_348289/932759980.py:2: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  train["d2"]=train['distance_est'].between(5, 10, inclusive=False)
/tmp/ipykernel_348289/932759980.py:3: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  train["d3"]=train['distance_est'].between(10, 20, inclusive=False)
/tmp/ipykernel_348289/932759980.py:4: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  train["d4"]=train['distance_est'].between(20, 30, inclusive=False)
/tmp/ipykernel_348289/932759980.py:5: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  train["d5"]=train['distance_est'].between(30, 100, inclusive=False)


In [17]:
#for test data
d_test = [distance(test['pickup_latitude'].values[i], test['pickup_longitude'].values[i],
              test['dropoff_latitude'].values[i],test['dropoff_longitude'].values[i]) for i in range(len(test['pickup_latitude']))]
test['distance_est']=d_test

## STEF-UPDATED

In [18]:
test["d1"]=test['distance_est']<=5
# test["d2"]=test['distance_est'].between(5, 10, inclusive=False)
# test["d3"]=test['distance_est'].between(10, 20, inclusive=False)
# test["d4"]=test['distance_est'].between(20, 30, inclusive=False)
# test["d5"]=test['distance_est'].between(30, 100, inclusive=False)
test["d2"]=test['distance_est'].between(5, 10, inclusive="neither")
test["d3"]=test['distance_est'].between(10, 20, inclusive="neither")
test["d4"]=test['distance_est'].between(20, 30, inclusive="neither")
test["d5"]=test['distance_est'].between(30, 100, inclusive="neither")
test["d6"]=test['distance_est']>100

/tmp/ipykernel_348289/2618483778.py:2: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  test["d2"]=test['distance_est'].between(5, 10, inclusive=False)
/tmp/ipykernel_348289/2618483778.py:3: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  test["d3"]=test['distance_est'].between(10, 20, inclusive=False)
/tmp/ipykernel_348289/2618483778.py:4: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  test["d4"]=test['distance_est'].between(20, 30, inclusive=False)
/tmp/ipykernel_348289/2618483778.py:5: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  test["d5"]=test['distance_est'].between(30, 100, inclusive=False)


In [19]:
#dimension reduction of pickup and dropoff (not sure about this step)
# ALEX: remove ML code
# from sklearn.decomposition import TruncatedSVD
# tsvd = TruncatedSVD(n_components=1, random_state=1)

In [20]:
#for train data
frames_p_train=[train['pickup_latitude'],train['pickup_longitude']]
p_train=pd.concat(frames_p_train,axis=1)

frames_d_train=[train['dropoff_latitude'],train['dropoff_longitude']]
d_train=pd.concat(frames_d_train,axis=1)

# ALEX: remove ML code
# train['tsvd_p']=tsvd.fit_transform(p_train)
# train['tsvd_d']=tsvd.fit_transform(d_train)
train['tsvd_p'] = p_train.iloc[:,0]
train['tsvd_d'] = d_train.iloc[:,0]

In [21]:
#for test data
frames_p_test=[test['pickup_latitude'],test['pickup_longitude']]
p_test=pd.concat(frames_p_test,axis=1)

frames_d_test=[test['dropoff_latitude'],test['dropoff_longitude']]
d_test=pd.concat(frames_d_test,axis=1)

# ALEX: remove ML code
# test['tsvd_p']=tsvd.fit_transform(p_test)
# test['tsvd_d']=tsvd.fit_transform(d_test)
test['tsvd_p'] = p_test.iloc[:,0]
test['tsvd_d'] = d_test.iloc[:,0]

In [22]:
import math

def calculate_initial_compass_bearing(pointA, pointB):
    if (type(pointA) != tuple) or (type(pointB) != tuple):
        raise TypeError("Only tuples are supported as arguments")

    lat1 = math.radians(pointA[0])
    lat2 = math.radians(pointB[0])

    diffLong = math.radians(pointB[1] - pointA[1])

    x = math.sin(diffLong) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1)
            * math.cos(lat2) * math.cos(diffLong))

    initial_bearing = math.atan2(x, y)

    # Now we have the initial bearing but math.atan2 return values
    # from -180° to + 180° which is not what we want for a compass bearing
    # The solution is to normalize the initial bearing as shown below
    initial_bearing = math.degrees(initial_bearing)
    compass_bearing = (initial_bearing + 360) % 360

    return compass_bearing

In [23]:
#for train data
train['direction']=[calculate_initial_compass_bearing((train['pickup_latitude'].loc[i],train['pickup_longitude'].loc[i]), 
                                  (train['dropoff_latitude'].loc[i],train['dropoff_longitude'].loc[i])) for i in range(len(train['pickup_latitude']))]

In [24]:
#for test data
test['direction']=[calculate_initial_compass_bearing((test['pickup_latitude'].loc[i],test['pickup_longitude'].loc[i]), 
                                  (test['dropoff_latitude'].loc[i],test['dropoff_longitude'].loc[i])) for i in range(len(test['pickup_latitude']))]

## STEF-UPDATED

In [25]:
# ALEX: remove plotting
# sns.heatmap(train.corr())
_ = train.corr(numeric_only=True)

/tmp/ipykernel_348289/308388489.py:3: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  _ = train.corr()


In [26]:
test_id=test["id"]
test = test.drop(['id', 'pickup_datetime'], axis=1)
X = train.drop(['id','trip_duration', 'pickup_datetime','dropoff_datetime'], axis=1)
Y = train['trip_duration']

In [27]:
# ALEX: remove ML code
# RF = RandomForestRegressor(verbose=True)
# RF.fit(X, Y)

In [28]:
features_list = X.columns.values
# ALEX: remove ML code
# feature_importance = RF.feature_importances_
# sorted_idx = np.argsort(feature_importance)
feature_importance = X.columns.values
sorted_idx = np.argsort(feature_importance)
 
# ALEX: remove plotting
# plt.figure(figsize=(5,10))
# plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align='center')
# plt.yticks(range(len(sorted_idx)), features_list[sorted_idx])
# plt.xlabel('Importance')
# plt.title('Feature importances')
# plt.draw()
# plt.show()
_ = feature_importance[sorted_idx]
_ = features_list[sorted_idx]

In [29]:
# ALEX: remove ML code
# Y_pred = RF.predict(test)
Y_pred = Y

In [30]:
sub = pd.DataFrame()
sub['id'] = test_id
sub['trip_duration'] = Y_pred
sub.to_csv('RF.csv', index=False)